# Main

In [1]:
# ======================
# 0. IMPROT
# ======================
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotInteractableException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import time
import re
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import pandas as pd
import signal
import sys
pd.set_option("display.max_columns", None)
# Global var untuk driver
driver = None

In [20]:
# ======================
# 3. IMPORT SPREADSHEET
# ======================

#new no kordinat
sbrdup = pd.read_csv("https://docs.google.com/spreadsheets/d/isi_sendiri_link_spreadsheetmu/export?format=csv&gid=1252202213")

mask = sbrdup['keberadaan_usaha'] == 1
# filter keep
#sbrdup = sbrdup[mask].reset_index(drop=True)
# filter remove
#sbrdup = sbrdup[~mask_petugas].reset_index(drop=True)

sbrdup["idsbr"] = (
    sbrdup["idsbr"]
    .astype(str)
    .str.replace(r"\.0$", "", regex=True)
    .replace("nan", pd.NA)
    .astype("Int64")  # integer nullable
)
sbrdup = sbrdup.iloc[0:500].reset_index(drop=True)
len(sbrdup)

500

# tandai duplikat dengan 1 dan 4 (Gilang Only)

In [13]:
def flag_duplicate(df):
    """
    Flag duplikat berdasarkan flag_dup dari detect_dup().
    Utama=1, duplikat=4 di kolom 'keberadaan_usaha'.
    Skip baris dengan flag_dup=0 (unik).
    """
    import pandas as pd
    import numpy as np
    
    if 'keberadaan_usaha' not in df.columns:
        df['keberadaan_usaha'] = np.nan
    
    # Hanya proses baris dengan flag_dup > 0
    dup_groups = df[df['flag_dup'] > 0].groupby('flag_dup')
    
    for dup_num, group in dup_groups:
        print(f"Grup dup #{dup_num}: {len(group)} rows")
        
        # 2. Hitung kolom terisi (non-null, non-empty str/num)
        def count_filled(row):
            return sum(1 for v in row if pd.notna(v) and str(v).strip() != '')
        
        group = group.copy()
        group['filled_cols'] = group.apply(count_filled, axis=1)
        
        max_filled = group['filled_cols'].max()
        candidates = group[group['filled_cols'] == max_filled]
        
        if len(candidates) == 1:
            # Utama: paling lengkap
            utama_idx = candidates.index[0]
        else:
            # 3. Hitung total char semua kolom
            def total_chars(row):
                return sum(len(str(v).strip()) for v in row if pd.notna(v))
            
            candidates = candidates.copy()
            candidates['total_chars'] = candidates.apply(total_chars, axis=1)
            max_chars = candidates['total_chars'].max()
            final_cand = candidates[candidates['total_chars'] == max_chars]
            
            if len(final_cand) == 1:
                utama_idx = final_cand.index[0]
            else:
                # 4. Index terkecil (baris pertama muncul)
                utama_idx = final_cand.index[0]
        
        # 5. Flag semua baris di group ini
        df.loc[group.index, 'keberadaan_usaha'] = 4  # Duplikat
        df.at[utama_idx, 'keberadaan_usaha'] = 1  # Utama
    
    dup_count = (df['keberadaan_usaha'] == 4).sum()
    utama_count = (df['keberadaan_usaha'] == 1).sum()
    print(f"\n✔ Flagged: {utama_count} utama, {dup_count} duplikat")
    print(f"  Baris unik (flag_dup=0): {(df['flag_dup'] == 0).sum()} (tidak diproses)")
    
    return df

# Usage (setelah detect_dup):
sbrdup = flag_duplicate(sbrdup)


Grup dup #1: 2 rows
Grup dup #2: 2 rows
Grup dup #3: 2 rows

✔ Flagged: 484 utama, 11 duplikat
  Baris unik (flag_dup=0): 494 (tidak diproses)


# fix latlong (Gilang Only)

In [8]:
# Fungsi untuk perbaiki format latlong dari text
def fix_latlong_format(value):
    if pd.isna(value):
        return None
    
    s = str(value).strip()
    if s == '' or s.lower() in ['nan', 'na', 'null']:
        return None
    
    # Ganti koma jadi titik (untuk locale Indonesia)
    s = s.replace(',', '.')
    
    # Hapus space/karakter aneh
    s = s.replace(' ', '')
    
    try:
        return float(s)
    except:
        return None

# Terapkan ke kolom latitude dan longitude
sbrdup["latitude"] = sbrdup["latitude"].apply(fix_latlong_format)
sbrdup["longitude"] = sbrdup["longitude"].apply(fix_latlong_format)

LAT_MIN, LAT_MAX = -2.358865559286161, -1.315090836134106
LON_MIN, LON_MAX = 115.13204822042515, 115.7495

def fix_latitude_decimal(lat):
    if pd.isna(lat):
        return lat
    
    # Kalau sudah dalam range Tabalong, skip
    if LAT_MIN <= lat <= LAT_MAX:
        return lat
    
    # Kalau positif atau terlalu besar/kecil, return as-is
    if lat >= 0 or lat < -10:
        return lat
    
    # Fix latitude yang salah tempat titiknya
    if -1 < lat < 0:  # antara 0 dan -1
        if lat > -0.01:  # -0.00xxx (geser 3 langkah)
            return lat * 1000
        elif lat > -0.1:  # -0.0xxx (geser 2 langkah)
            return lat * 100
        else:  # -0.xxx (geser 1 langkah)
            return lat * 10
    
    return lat

# Apply fix
sbrdup['latitude'] = sbrdup['latitude'].apply(fix_latitude_decimal)

# Filter (CUMA SEKALI)
mask_in = (
    sbrdup["latitude"].notna() &
    sbrdup["longitude"].notna() &
    sbrdup["latitude"].between(LAT_MIN, LAT_MAX, inclusive="both") &
    sbrdup["longitude"].between(LON_MIN, LON_MAX, inclusive="both")
)

# Simpan outlier
outlier = sbrdup.loc[~mask_in].copy()
#sbrdup.to_excel(r"C:\Users\ACER\Downloads\tambahan3453.xlsx", index=False)

# Keep yang di dalam bbox
#sbrdup = sbrdup.loc[mask_in].copy()

# kosongkan yang ada di dalam bounding box
sbrdup.loc[~mask_in, ["latitude", "longitude"]] = pd.NA

print("outlier:", len(outlier))
print("sisa:", len(sbrdup))
print(f"Latitude range: {sbrdup['latitude'].min():.6f} to {sbrdup['latitude'].max():.6f}")
print(f"Longitude range: {sbrdup['longitude'].min():.6f} to {sbrdup['longitude'].max():.6f}")
print(f"NaN lat: {sbrdup['latitude'].isna().sum()}")
print(f"NaN lon: {sbrdup['longitude'].isna().sum()}")


outlier: 500
sisa: 500
Latitude range: nan to nan
Longitude range: nan to nan
NaN lat: 500
NaN lon: 500


# Detect Duplicate (Gilang Only)

In [12]:
def detect_dup(df):
    """
    Deteksi duplikat berdasarkan nama_usaha (normalized) + nmdesa + nmkec.
    Isi kolom 'norm_nama_usaha' dan 'flag_dup' (nomor kelompok duplikat).
    """
    import re
    import pandas as pd
    
    # 1) Normalisasi nama_usaha: lowercase, hapus non-alphanumeric, sort kata
    def normalize_name(name):
        if pd.isna(name):
            return ""
        # Lowercase + hapus non-alphanumeric
        clean = re.sub(r'[^a-z0-9\s]', '', str(name).lower())
        # Split kata, sort alfabetis, join kembali (supaya urutan kata ga pengaruh)
        words = sorted(clean.split())
        return ' '.join(words)
    
    df['norm_nama_usaha'] = df['nama_usaha'].apply(normalize_name)
    
    # 2) Inisialisasi flag_dup
    df['flag_dup'] = 0
    
    # 3) Deteksi duplikat: norm_nama_usaha + nmdesa + nmkec sama
    # Group by normalized name, desa, kec
    grouped = df.groupby(['norm_nama_usaha', 'nmdesa', 'nmkec'], dropna=False)
    
    dup_group_num = 1
    
    for (norm_name, desa, kec), group_df in grouped:
        # Skip kelompok dengan 1 baris saja (bukan duplikat)
        if len(group_df) <= 1:
            continue
        
        # Skip kalau norm_name kosong (NaN nama_usaha)
        if not norm_name or norm_name.strip() == "":
            continue
        
        # Tandai semua baris di kelompok ini dengan nomor duplikat
        df.loc[group_df.index, 'flag_dup'] = dup_group_num
        dup_group_num += 1
    
    total_dup = (df['flag_dup'] > 0).sum()
    total_groups = df['flag_dup'].max()
    
    print(f"✔ Deteksi duplikat selesai:")
    print(f"  - {total_dup} baris duplikat")
    print(f"  - {total_groups} kelompok duplikat")
    
    return df

sbrdup = detect_dup(sbrdup)


✔ Deteksi duplikat selesai:
  - 6 baris duplikat
  - 3 kelompok duplikat


In [80]:
#sbrdup = sbrdup.iloc[26166:].reset_index(drop=True)
sbrdup.to_excel(r"C:\Users\ACER\Downloads\sbrdup.xlsx", index=False)
len(sbrdup)


3306

# Isi dan Match Desa dan Kecamatan (Gilang Only)

## 1. kdkec dan kddesa terisi

In [9]:
# Fix tipe nullable dulu (hindari IntCastingNaNError)
msls = pd.read_excel(r"C:\Users\ACER\Downloads\msls_25_1_6309.xlsx")
for df in (sbrdup, msls):
    df["kdkec"] = pd.to_numeric(df["kdkec"], errors='coerce').astype("Int64")
    df["kddesa"] = pd.to_numeric(df["kddesa"], errors='coerce').astype("Int64")

# Buat ref UNIK per (kdkec, kddesa) dari msls
ref = msls[["kdkec", "kddesa", "nmkec", "nmdesa"]].drop_duplicates(subset=["kdkec", "kddesa"])

print("msls unik desa:", ref.shape[0])
print("sbrdup rows sebelum:", sbrdup.shape[0])

# Merge left: tambah kolom, no row tambah/duplikat
sbrdup = sbrdup.merge(ref, on=["kdkec", "kddesa"], how="left", suffixes=('', '_ref'))

print("sbrdup rows sesudah:", sbrdup.shape[0])  # Harus sama!
print("Match rate:", (sbrdup["nmkec"].notna()).mean() * 100, "%")

msls unik desa: 131
sbrdup rows sebelum: 500
sbrdup rows sesudah: 500
Match rate: 0.0 %


## 2. kdkec dan kddesa kosong, lihat di string alamat

In [10]:
# Siapkan list referensi dari msls (drop duplicates untuk efisiensi)
ref_desa = msls[['nmdesa', 'nmkec', 'kdkec', 'kddesa']].drop_duplicates(subset=['nmdesa'])
ref_kec = msls[['nmkec', 'kdkec']].drop_duplicates(subset=['nmkec'])

# Fungsi untuk cari string desa di alamat
def cari_desa_di_alamat(alamat, ref_desa_df):
    if pd.isna(alamat) or alamat.strip() == "":
        return None, None, None, None
    
    alamat_lower = str(alamat).lower()
    
    # Loop semua nama desa di msls
    for _, row in ref_desa_df.iterrows():
        desa_lower = str(row['nmdesa']).lower()
        
        # Cek apakah nama desa ada dalam alamat
        if desa_lower in alamat_lower:
            return row['nmdesa'], row['nmkec'], row['kdkec'], row['kddesa']
    
    return None, None, None, None

# Fungsi untuk cari string kecamatan di alamat
def cari_kec_di_alamat(alamat, ref_kec_df):
    if pd.isna(alamat) or alamat.strip() == "":
        return None, None
    
    alamat_lower = str(alamat).lower()
    
    # Loop semua nama kecamatan di msls
    for _, row in ref_kec_df.iterrows():
        kec_lower = str(row['nmkec']).lower()
        
        # Cek apakah nama kecamatan ada dalam alamat
        if kec_lower in alamat_lower:
            return row['nmkec'], row['kdkec']
    
    return None, None

# Inisialisasi kolom baru jika belum ada
if 'nmdesa' not in sbrdup.columns:
    sbrdup['nmdesa'] = np.nan
if 'nmkec' not in sbrdup.columns:
    sbrdup['nmkec'] = np.nan
if 'kdkec' not in sbrdup.columns:
    sbrdup['kdkec'] = pd.NA
if 'kddesa' not in sbrdup.columns:
    sbrdup['kddesa'] = pd.NA

print(f"Total baris sbrdup: {len(sbrdup)}")
filled_desa = 0
filled_kec_only = 0
not_found = 0

# Iterasi setiap baris sbrdup
for idx, row in sbrdup.iterrows():
    # Skip jika nmdesa dan nmkec sudah terisi
    if pd.notna(row.get('nmdesa')) and pd.notna(row.get('nmkec')):
        continue
    
    alamat = row.get('alamat', '')
    
    # LANGKAH 1: Cari desa di alamat
    desa_match, kec_match, kdkec_match, kddesa_match = cari_desa_di_alamat(alamat, ref_desa)
    
    if desa_match:
        sbrdup.at[idx, 'nmdesa'] = desa_match
        sbrdup.at[idx, 'nmkec'] = kec_match
        sbrdup.at[idx, 'kdkec'] = kdkec_match
        sbrdup.at[idx, 'kddesa'] = kddesa_match
        filled_desa += 1
        print(f"  ✅ Baris {idx}: Desa '{desa_match}', Kec '{kec_match}'")
        continue
    
    # LANGKAH 2: Jika gagal, cari hanya kecamatan
    kec_match, kdkec_match = cari_kec_di_alamat(alamat, ref_kec)
    
    if kec_match:
        sbrdup.at[idx, 'nmkec'] = kec_match
        sbrdup.at[idx, 'kdkec'] = kdkec_match
        filled_kec_only += 1
        print(f"  ⚠️  Baris {idx}: Hanya Kec '{kec_match}' (desa tidak ketemu)")
        continue
    
    # LANGKAH 3: Tidak ketemu sama sekali, kosongkan
    sbrdup.at[idx, 'nmdesa'] = np.nan
    sbrdup.at[idx, 'nmkec'] = np.nan
    not_found += 1
    print(f"  ❌ Baris {idx}: Tidak ketemu")

print(f"\n📊 Summary:")
print(f"  - Berhasil isi desa + kec: {filled_desa}")
print(f"  - Berhasil isi kec saja: {filled_kec_only}")
print(f"  - Tidak ketemu: {not_found}")

Total baris sbrdup: 500
  ✅ Baris 0: Desa 'TANJUNG', Kec 'TANJUNG'
  ✅ Baris 1: Desa 'TANJUNG', Kec 'TANJUNG'
  ✅ Baris 2: Desa 'TANJUNG', Kec 'TANJUNG'
  ✅ Baris 3: Desa 'BATANG BANYU', Kec 'BANUA LAWAS'
  ❌ Baris 4: Tidak ketemu
  ❌ Baris 5: Tidak ketemu
  ❌ Baris 6: Tidak ketemu
  ✅ Baris 7: Desa 'KAMBITIN', Kec 'TANJUNG'
  ✅ Baris 8: Desa 'KAMBITIN', Kec 'TANJUNG'
  ✅ Baris 9: Desa 'PEMBATAAN', Kec 'MURUNG PUDAK'
  ✅ Baris 10: Desa 'USIH', Kec 'BINTANG ARA'
  ✅ Baris 11: Desa 'HALONG', Kec 'HARUAI'
  ✅ Baris 12: Desa 'JARO', Kec 'JARO'
  ✅ Baris 13: Desa 'PULAU', Kec 'KELUA'
  ✅ Baris 14: Desa 'TANTARINGIN', Kec 'MUARA HARUS'
  ✅ Baris 15: Desa 'MUARA UYA', Kec 'MUARA UYA'
  ❌ Baris 16: Tidak ketemu
  ✅ Baris 17: Desa 'PUGAAN', Kec 'PUGAAN'
  ✅ Baris 18: Desa 'TANJUNG', Kec 'TANJUNG'
  ✅ Baris 19: Desa 'TANTA', Kec 'TANTA'
  ✅ Baris 20: Desa 'PANGELAK', Kec 'UPAU'
  ✅ Baris 21: Desa 'TANJUNG', Kec 'TANJUNG'
  ✅ Baris 22: Desa 'PEMBATAAN', Kec 'MURUNG PUDAK'
  ✅ Baris 23: Desa 'TANJ

C:\Users\ACER\AppData\Local\Temp\ipykernel_10800\470998471.py:66: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'TANJUNG' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sbrdup.at[idx, 'nmdesa'] = desa_match


  ✅ Baris 42: Desa 'TANJUNG', Kec 'TANJUNG'
  ✅ Baris 43: Desa 'AGUNG', Kec 'TANJUNG'
  ✅ Baris 44: Desa 'AGUNG', Kec 'TANJUNG'
  ✅ Baris 45: Desa 'TANJUNG', Kec 'TANJUNG'
  ❌ Baris 46: Tidak ketemu
  ✅ Baris 47: Desa 'PADANGIN', Kec 'MUARA HARUS'
  ✅ Baris 48: Desa 'SULINGAN', Kec 'MURUNG PUDAK'
  ✅ Baris 49: Desa 'SULINGAN', Kec 'MURUNG PUDAK'
  ⚠️  Baris 50: Hanya Kec 'MURUNG PUDAK' (desa tidak ketemu)
  ✅ Baris 51: Desa 'TANJUNG', Kec 'TANJUNG'
  ✅ Baris 52: Desa 'KAMPUNG BARU', Kec 'MUARA UYA'
  ✅ Baris 53: Desa 'TANJUNG', Kec 'TANJUNG'
  ❌ Baris 54: Tidak ketemu
  ❌ Baris 55: Tidak ketemu
  ✅ Baris 56: Desa 'KAMPUNG BARU', Kec 'MUARA UYA'
  ✅ Baris 57: Desa 'MABURAI', Kec 'MURUNG PUDAK'
  ✅ Baris 58: Desa 'NAWIN', Kec 'HARUAI'
  ❌ Baris 59: Tidak ketemu
  ❌ Baris 60: Tidak ketemu
  ✅ Baris 61: Desa 'BELIMBING RAYA', Kec 'MURUNG PUDAK'
  ✅ Baris 62: Desa 'TANTA', Kec 'TANTA'
  ❌ Baris 63: Tidak ketemu
  ❌ Baris 64: Tidak ketemu
  ❌ Baris 65: Tidak ketemu
  ❌ Baris 66: Tidak ketemu

In [56]:
#msls.to_excel(r"C:\Users\ACER\Downloads\msls.xlsx", index=False)
sbrdup.to_excel(r"C:\Users\ACER\Downloads\sbrdup.xlsx", index=False)


# fungsi isi latlong kosong (fungsinya doang, running nya di bagian USAGE)

## tahap 1

In [28]:
def isi_latlong(
    df,
    col_nmusaha,
    col_alamat,
    col_nmdesa,
    col_nmkec,
    col_lat,
    col_lon,
    col_gmaps,
    sleep_time=0.5
):
    df[col_gmaps] = df[col_gmaps].astype("object")

    LAT_MIN, LAT_MAX = -2.358865559286161, -1.315090836134106
    LON_MIN, LON_MAX = 115.13204822042515, 115.7495

    def _missing(x):
        if pd.isna(x):
            return True
        if isinstance(x, str):
            s = x.strip().lower()
            return s == "" or s == "nan" or s == "na" or s == "null"
        return False


    def new_driver():
        options = Options()
        options.add_argument("--window-size=1920,1080")
        options.add_argument("--headless=new")

        options.page_load_strategy = "eager"
        prefs = {
            "profile.managed_default_content_settings.images": 2,
            "profile.default_content_setting_values.notifications": 2,
            "profile.managed_default_content_settings.stylesheets": 2,
        }
        options.add_experimental_option("prefs", prefs)
        options.add_argument("--disable-gpu")
        options.add_argument("--disable-extensions")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--no-sandbox")
        options.add_argument("--blink-settings=imagesEnabled=false")
        options.add_argument("--disable-blink-features=AutomationControlled")

        driver = webdriver.Chrome(options=options)

        driver.execute_cdp_cmd(
            "Page.addScriptToEvaluateOnNewDocument",
            {"source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"}
        )

        driver.execute_cdp_cmd(
            "Emulation.setGeolocationOverride",
            {"latitude": -1.71, "longitude": 115.28, "accuracy": 100}
        )

        driver.get("https://www.google.com/maps")
        wait = WebDriverWait(driver, 5)
        time.sleep(1.5)
        return driver, wait

    def safe_quit(driver):
        try:
            driver.quit()  # recommended untuk mengakhiri session [web:19]
        except Exception:
            pass

    def restart_and_skip(driver, wait, i, total, reason, mark=None):
        print(f"  ❌ {reason} -> pencarian jalur terpicu. isi manual baris ini | baris {i+1}/{total}")
        if mark:
            df.at[i, col_gmaps] = mark
        safe_quit(driver)
        driver, wait = new_driver()
        return driver, wait, i + 1  # skip baris ini

    driver, wait = new_driver()

    total = len(df)
    processed = 0
    skipped = 0
    interrupted = False
    i = 0

    try:
        while i < total and not interrupted:
            r = df.iloc[i]

            # 1) skip kalau sudah ada latlon
            if pd.notna(r[col_lat]) and pd.notna(r[col_lon]):
                print(f"▶ {i+1}/{total}: SKIP (sudah ada latlon)")
                skipped += 1
                i += 1
                continue
        
            # 2) skip total kalau keberadaan_usaha != 1
            if r["keberadaan_usaha"] != 1:
                print(f"▶ {i+1}/{total}: SKIP TOTAL (keberadaan_usaha != 1)")
                skipped += 1
                i += 1
                continue
                
            # ===== TAMBAHKAN BLOK INI =====
            desa_raw = r[col_nmdesa]
            kec_raw = r[col_nmkec]
            
            allow_admin = not (_missing(desa_raw) or _missing(kec_raw))
            # ===== SAMPAI SINI =====
            nmusaha = str(r[col_nmusaha]).strip()
            alamat = str(r[col_alamat]).strip()

            nmdesa = "" if _missing(desa_raw) else str(desa_raw).strip()
            nmkec = "" if _missing(kec_raw) else str(kec_raw).strip()

            print(f"▶ {i+1}/{total}: {nmusaha} - {nmdesa} {nmkec}")
            processed += 1

            found_usaha = False

            # =========================================================
            # 0) COBA CARI NMUSAHA DOANG
            # =========================================================
            q_usaha = f"usaha {nmusaha} tabalong"
            try:
                box = wait.until(EC.presence_of_element_located((By.ID, "UGojuc")))
                box.clear()
                box.send_keys(q_usaha + Keys.ENTER)

                try:
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.hfpxzc")))
                    time.sleep(1)
                except Exception:
                    time.sleep(2)

                url = driver.current_url

                if "/place/" in url:
                    m = re.search(r"@(-?\d+\.\d+),(-?\d+\.\d+)", url)
                    if m:
                        lat = float(m.group(1))
                        lon = float(m.group(2))

                        try:
                            result_name = driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf").text.strip()
                        except Exception:
                            result_name = driver.title.split(" - ")[0].strip()

                        words_usaha = set(w.lower() for w in nmusaha.split())
                        words_result = set(w.lower() for w in result_name.split())
                        match_count = len(words_usaha & words_result)

                        if match_count >= 1 and LAT_MIN <= lat <= LAT_MAX and LON_MIN <= lon <= LON_MAX:
                            df.at[i, col_lat] = lat
                            df.at[i, col_lon] = lon
                            df.at[i, col_gmaps] = result_name
                            print(f"  ✅ DIRECT USAHA ({match_count} kata match): {lat:.5f}, {lon:.5f} | {result_name[:40]}")
                            found_usaha = True
                        else:
                            print(f"  ⚠ Direct point match {match_count} kata (min 1) atau di luar Tabalong")

                else:
                    try:
                        results = driver.find_elements(By.CSS_SELECTOR, "a.hfpxzc")
                        for idx, result in enumerate(results[:5]):
                            try:
                                aria_label = result.get_attribute("aria-label")
                                if not aria_label:
                                    continue
                                place_name = aria_label.split("·")[0].split("•")[0].strip()

                                words_usaha = set(w.lower() for w in nmusaha.split())
                                words_place = set(w.lower() for w in place_name.split())
                                match_count = len(words_usaha & words_place)

                                if match_count >= 2:
                                    result.click()
                                    time.sleep(2)

                                    url = driver.current_url
                                    m = re.search(r"@(-?\d+\.\d+),(-?\d+\.\d+)", url)
                                    if m:
                                        lat = float(m.group(1))
                                        lon = float(m.group(2))

                                        if LAT_MIN <= lat <= LAT_MAX and LON_MIN <= lon <= LON_MAX:
                                            try:
                                                final_name = driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf").text.strip()
                                            except Exception:
                                                final_name = place_name

                                            df.at[i, col_lat] = lat
                                            df.at[i, col_lon] = lon
                                            df.at[i, col_gmaps] = final_name

                                            print(f"  ✅ LIST USAHA #{idx+1} ({match_count} kata match): {lat:.5f}, {lon:.5f} | {final_name[:40]}")
                                            found_usaha = True
                                            break
                                        else:
                                            print(f"  ⚠ List #{idx+1} di luar Tabalong")
                                            driver.back()
                                            time.sleep(1)
                            except Exception:
                                continue

                        if not found_usaha:
                            print("  ⚠ Tidak ada hasil pencarian langsung")
                    except Exception:
                        print("  ⚠ Tidak ada hasil pencarian langsung")

            except ElementNotInteractableException:
                driver, wait, i = restart_and_skip(
                    driver, wait, i, total,
                    reason="ElementNotInteractableException (logika 1)",
                    mark="MANUAL - ElementNotInteractable (logika 0)"
                )
                continue
            except WebDriverException as e:
                driver, wait, i = restart_and_skip(
                    driver, wait, i, total,
                    reason=f"WebDriverException (logika 1): {str(e)[:80]}",
                    mark="MANUAL - WebDriverException (logika 0)"
                )
                continue
            except Exception as e:
                print(f"  ❌ Error cari usaha: {str(e)[:50]}")
            # kalau desa/kec kosong, stop di sini (jangan lanjut logika 1-3)
            if not allow_admin:
                print(f"▶ {i+1}/{total}: STOP setelah logika 0 (desa/kec kosong)")
                time.sleep(sleep_time)
                i += 1
                continue  # skip ke baris berikutnya
            # =========================================================
            # 1) COBA CARI NMUSAHA + DESA + KEC
            # =========================================================
            q_usaha = f"usaha {nmusaha} desa {nmdesa} kecamatan {nmkec} kabupaten tabalong"
            try:
                box = wait.until(EC.presence_of_element_located((By.ID, "UGojuc")))
                box.clear()
                box.send_keys(q_usaha + Keys.ENTER)

                try:
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.hfpxzc")))
                    time.sleep(1)
                except Exception:
                    time.sleep(2)

                url = driver.current_url

                if "/place/" in url:
                    m = re.search(r"@(-?\d+\.\d+),(-?\d+\.\d+)", url)
                    if m:
                        lat = float(m.group(1))
                        lon = float(m.group(2))

                        try:
                            result_name = driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf").text.strip()
                        except Exception:
                            result_name = driver.title.split(" - ")[0].strip()

                        words_usaha = set(w.lower() for w in nmusaha.split())
                        words_result = set(w.lower() for w in result_name.split())
                        match_count = len(words_usaha & words_result)

                        if match_count >= 1 and LAT_MIN <= lat <= LAT_MAX and LON_MIN <= lon <= LON_MAX:
                            df.at[i, col_lat] = lat
                            df.at[i, col_lon] = lon
                            df.at[i, col_gmaps] = result_name
                            print(f"  ✅ DIRECT USAHA ({match_count} kata match): {lat:.5f}, {lon:.5f} | {result_name[:40]}")
                            found_usaha = True
                        else:
                            print(f"  ⚠ Direct point match {match_count} kata (min 2) atau di luar Tabalong")

                else:
                    try:
                        results = driver.find_elements(By.CSS_SELECTOR, "a.hfpxzc")
                        for idx, result in enumerate(results[:5]):
                            try:
                                aria_label = result.get_attribute("aria-label")
                                if not aria_label:
                                    continue
                                place_name = aria_label.split("·")[0].split("•")[0].strip()

                                words_usaha = set(w.lower() for w in nmusaha.split())
                                words_place = set(w.lower() for w in place_name.split())
                                match_count = len(words_usaha & words_place)

                                if match_count >= 2:
                                    result.click()
                                    time.sleep(2)

                                    url = driver.current_url
                                    m = re.search(r"@(-?\d+\.\d+),(-?\d+\.\d+)", url)
                                    if m:
                                        lat = float(m.group(1))
                                        lon = float(m.group(2))

                                        if LAT_MIN <= lat <= LAT_MAX and LON_MIN <= lon <= LON_MAX:
                                            try:
                                                final_name = driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf").text.strip()
                                            except Exception:
                                                final_name = place_name

                                            df.at[i, col_lat] = lat
                                            df.at[i, col_lon] = lon
                                            df.at[i, col_gmaps] = final_name

                                            print(f"  ✅ LIST USAHA #{idx+1} ({match_count} kata match): {lat:.5f}, {lon:.5f} | {final_name[:40]}")
                                            found_usaha = True
                                            break
                                        else:
                                            print(f"  ⚠ List #{idx+1} di luar Tabalong")
                                            driver.back()
                                            time.sleep(1)
                            except Exception:
                                continue

                        if not found_usaha:
                            print("  ⚠ Tidak ada hasil list match ≥2 kata, coba alamat")
                    except Exception:
                        print("  ⚠ Tidak ada hasil usaha, coba alamat")

            except ElementNotInteractableException:
                driver, wait, i = restart_and_skip(
                    driver, wait, i, total,
                    reason="ElementNotInteractableException (logika 1)",
                    mark="MANUAL - ElementNotInteractable (logika 1)"
                )
                continue
            except WebDriverException as e:
                driver, wait, i = restart_and_skip(
                    driver, wait, i, total,
                    reason=f"WebDriverException (logika 1): {str(e)[:80]}",
                    mark="MANUAL - WebDriverException (logika 1)"
                )
                continue
            except Exception as e:
                print(f"  ❌ Error cari usaha: {str(e)[:50]}")


            # =========================================================
            # 2) KALAU GAGAL, COBA NMUSAHA + ALAMAT
            # =========================================================
            if not found_usaha:
                q_alamat = f"{nmusaha} {alamat} desa {nmdesa} kecamatan {nmkec} kabupaten tabalong"
                try:
                    box = wait.until(EC.presence_of_element_located((By.ID, "UGojuc")))
                    box.clear()
                    box.send_keys(q_alamat + Keys.ENTER)

                    try:
                        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.hfpxzc")))
                        time.sleep(1)
                    except Exception:
                        time.sleep(2)

                    url = driver.current_url

                    if "/place/" in url:
                        m = re.search(r"@(-?\d+\.\d+),(-?\d+\.\d+)", url)
                        if m:
                            lat = float(m.group(1))
                            lon = float(m.group(2))

                            try:
                                result_name = driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf").text.strip()
                            except Exception:
                                result_name = driver.title.split(" - ")[0].strip()

                            words_usaha = set(w.lower() for w in nmusaha.split())
                            words_result = set(w.lower() for w in result_name.split())
                            match_count = len(words_usaha & words_result)

                            if match_count >= 1 and LAT_MIN <= lat <= LAT_MAX and LON_MIN <= lon <= LON_MAX:
                                df.at[i, col_lat] = lat
                                df.at[i, col_lon] = lon
                                df.at[i, col_gmaps] = result_name
                                print(f"  ✅ DIRECT ALAMAT ({match_count} kata match): {lat:.5f}, {lon:.5f} | {result_name[:40]}")
                                found_usaha = True
                            else:
                                print(f"  ⚠ Direct alamat match {match_count} kata (min 2) atau di luar Tabalong")

                    else:
                        try:
                            results = driver.find_elements(By.CSS_SELECTOR, "a.hfpxzc")
                            for idx, result in enumerate(results[:5]):
                                try:
                                    aria_label = result.get_attribute("aria-label")
                                    if not aria_label:
                                        continue
                                    place_name = aria_label.split("·")[0].split("•")[0].strip()

                                    words_usaha = set(w.lower() for w in nmusaha.split())
                                    words_place = set(w.lower() for w in place_name.split())
                                    match_count = len(words_usaha & words_place)

                                    if match_count >= 2:
                                        result.click()
                                        time.sleep(2)

                                        url = driver.current_url
                                        m = re.search(r"@(-?\d+\.\d+),(-?\d+\.\d+)", url)
                                        if m:
                                            lat = float(m.group(1))
                                            lon = float(m.group(2))

                                            if LAT_MIN <= lat <= LAT_MAX and LON_MIN <= lon <= LON_MAX:
                                                try:
                                                    final_name = driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf").text.strip()
                                                except Exception:
                                                    final_name = place_name

                                                df.at[i, col_lat] = lat
                                                df.at[i, col_lon] = lon
                                                df.at[i, col_gmaps] = final_name
                                                print(f"  ✅ LIST ALAMAT #{idx+1} ({match_count} kata match): {lat:.5f}, {lon:.5f} | {final_name[:40]}")
                                                found_usaha = True
                                                break
                                            else:
                                                print(f"  ⚠ List alamat #{idx+1} di luar Tabalong")
                                                driver.back()
                                                time.sleep(1)
                                except Exception:
                                    continue

                            if not found_usaha:
                                print("  ⚠ Tidak ada hasil list alamat match ≥2 kata, coba kantor desa")
                        except Exception:
                            print("  ⚠ Tidak ada hasil alamat, coba kantor desa")

                except ElementNotInteractableException:
                    driver, wait, i = restart_and_skip(
                        driver, wait, i, total,
                        reason="ElementNotInteractableException (logika 2)",
                        mark="MANUAL - ElementNotInteractable (logika 2)"
                    )
                    continue
                except WebDriverException as e:
                    driver, wait, i = restart_and_skip(
                        driver, wait, i, total,
                        reason=f"WebDriverException (logika 2): {str(e)[:80]}",
                        mark="MANUAL - WebDriverException (logika 2)"
                    )
                    continue
                except Exception as e:
                    print(f"  ❌ Error cari alamat: {str(e)[:50]}")

            # =========================================================
            # 3) CEK DF BARIS SEBELUMNYA ATAU KANTOR DESA / DEFAULT
            # =========================================================
            if not found_usaha:
                if _missing(r[col_nmdesa]) and _missing(r[col_nmkec]):
                    df.at[i, col_lat] = -2.1644076741997087
                    df.at[i, col_lon] = 115.38251770091627
                    df.at[i, col_gmaps] = "Kantor Bupati Tabalong"
                    print("  ✅ DEFAULT BUPATI (desa & kec kosong): -2.16441, 115.38252 | Kantor Bupati Tabalong")

                else:
                    prev_row = df.loc[:i-1][
                        (df.loc[:i-1, col_nmdesa] == nmdesa) &
                        (df.loc[:i-1, col_nmkec] == nmkec) &
                        (df.loc[:i-1, col_lat].notna()) &
                        (df.loc[:i-1, col_lon].notna())
                    ]

                    found_copy = False
                    if len(prev_row) > 0:
                        for prev_idx in prev_row.index:
                            prev_gmaps = str(df.at[prev_idx, col_gmaps]).lower()
                            words_desa = set(w.lower() for w in nmdesa.split())
                            words_gmaps = set(w.lower() for w in prev_gmaps.split())
                            match_desa = len(words_desa & words_gmaps)

                            if match_desa >= 1:
                                df.at[i, col_lat] = df.at[prev_idx, col_lat]
                                df.at[i, col_lon] = df.at[prev_idx, col_lon]
                                df.at[i, col_gmaps] = df.at[prev_idx, col_gmaps]
                                print(f"  ✅ COPAS DESA (baris {prev_idx+1}): {df.at[i, col_lat]:.5f}, {df.at[i, col_lon]:.5f} | {str(df.at[i, col_gmaps])[:40]}")
                                found_copy = True
                                break

                        if not found_copy:
                            print(f"  ⚠ Ada {len(prev_row)} baris desa sama, tapi gmaps nama tidak cocok")

                    if not found_copy:
                        q_desa = f"Kantor Desa {nmdesa} kecamatan {nmkec} kabupaten tabalong"
                        try:
                            box = wait.until(EC.presence_of_element_located((By.ID, "UGojuc")))
                            box.clear()
                            box.send_keys(q_desa + Keys.ENTER)

                            try:
                                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.hfpxzc")))
                                time.sleep(1)
                            except Exception:
                                time.sleep(2)

                            url = driver.current_url
                            m = re.search(r"@(-?\d+\.\d+),(-?\d+\.\d+)", url)
                            if m:
                                lat = float(m.group(1))
                                lon = float(m.group(2))

                                if LAT_MIN <= lat <= LAT_MAX and LON_MIN <= lon <= LON_MAX:
                                    nama = driver.title.split(" - ")[0].strip() if " - " in driver.title else f"Kantor Desa {nmdesa}"
                                    df.at[i, col_lat] = lat
                                    df.at[i, col_lon] = lon
                                    df.at[i, col_gmaps] = nama
                                    print(f"  ✅ DESA: {lat:.5f}, {lon:.5f} | {nama[:40]}")
                                else:
                                    print(f"  ❌ Kantor desa di luar Tabalong: lat={lat:.5f}, lon={lon:.5f}")
                            else:
                                print("  ❌ No coord in URL")

                        except ElementNotInteractableException:
                            driver, wait, i = restart_and_skip(
                                driver, wait, i, total,
                                reason="ElementNotInteractableException (logika 3 - kantor desa)",
                                mark="MANUAL - ElementNotInteractable (logika 3)"
                            )
                            continue
                        except WebDriverException as e:
                            driver, wait, i = restart_and_skip(
                                driver, wait, i, total,
                                reason=f"WebDriverException (logika 3 - kantor desa): {str(e)[:80]}",
                                mark="MANUAL - WebDriverException (logika 3)"
                            )
                            continue
                        except Exception as e:
                            print(f"  ❌ Error kantor desa: {str(e)[:50]}")

            time.sleep(sleep_time)
            i += 1

    except KeyboardInterrupt:
        print("\n⚠ Progres diinterupsi! Menyimpan progress...")
        interrupted = True
    finally:
        safe_quit(driver)
        print(f"\n✔ Processed: {processed}, Skipped: {skipped}, Total selesai: {i}/{total}")

    return df


## tahap 2 (kalo gagal gara2 nmdesa dan/atau nmkec kosong)
tahap 2 harus lewat match desa-kecamatan no 2 dulu. btw fungsinya bapuk (selalu gagal return padahal udah kuset tidak boleh gagal) dan aku males benerin, mending gausah dipake

In [9]:
def isi_latlongdua(
    df,
    col_nmusaha,
    col_alamat,
    col_nmdesa,
    col_nmkec,
    col_lat,
    col_lon,
    col_gmaps,
    sleep_time=0.5
):
    df[col_gmaps] = df[col_gmaps].astype("object")

    LAT_MIN, LAT_MAX = -2.358865559286161, -1.315090836134106
    LON_MIN, LON_MAX = 115.13204822042515, 115.7495

    def _missing(x):
        if pd.isna(x):
            return True
        if isinstance(x, str):
            s = x.strip().lower()
            return s == "" or s == "nan" or s == "na" or s == "null"
        return False

    def new_driver():
        options = Options()
        options.add_argument("--window-size=1920,1080")
        options.add_argument("--headless=new")
        options.page_load_strategy = "eager"
        
        prefs = {
            "profile.managed_default_content_settings.images": 2,
            "profile.default_content_setting_values.notifications": 2,
            "profile.managed_default_content_settings.stylesheets": 2,
        }
        options.add_experimental_option("prefs", prefs)
        options.add_argument("--disable-gpu")
        options.add_argument("--disable-extensions")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--no-sandbox")
        options.add_argument("--blink-settings=imagesEnabled=false")
        options.add_argument("--disable-blink-features=AutomationControlled")

        driver = webdriver.Chrome(options=options)

        driver.execute_cdp_cmd(
            "Page.addScriptToEvaluateOnNewDocument",
            {"source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"}
        )

        driver.execute_cdp_cmd(
            "Emulation.setGeolocationOverride",
            {"latitude": -1.71, "longitude": 115.28, "accuracy": 100}
        )

        driver.get("https://www.google.com/maps")
        wait = WebDriverWait(driver, 5)
        time.sleep(1.5)
        return driver, wait

    def safe_quit(driver):
        try:
            driver.quit()
        except Exception:
            pass

    def restart_and_skip(driver, wait, i, total, reason, mark=None):
        print(f"  ❌ {reason} -> restart browser | baris {i+1}/{total}")
        if mark:
            df.at[i, col_gmaps] = mark
        safe_quit(driver)
        driver, wait = new_driver()
        return driver, wait, i + 1

    def extract_coordinates(url):
        """Ekstrak lat/lon dari URL Google Maps"""
        m = re.search(r"@(-?\d+\.\d+),(-?\d+\.\d+)", url)
        if m:
            return float(m.group(1)), float(m.group(2))
        return None, None

    def is_in_tabalong(lat, lon):
        """Cek apakah koordinat dalam bounding box Tabalong"""
        if lat is None or lon is None:
            return False
        return LAT_MIN <= lat <= LAT_MAX and LON_MIN <= lon <= LON_MAX

    def get_place_address(driver):
        """Ambil alamat lengkap dari detail place"""
        try:
            address_elem = driver.find_element(By.CSS_SELECTOR, "div.Io6YTe.fontBodyMedium.kR99db.fdkmkc")
            return address_elem.text.strip()
        except Exception:
            return ""

    def is_direct_point(driver):
        """Cek apakah halaman menampilkan direct point (ada elemen h1.DUwDvf)"""
        try:
            driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf")
            return True
        except Exception:
            return False

    driver, wait = new_driver()

    total = len(df)
    processed = 0
    skipped = 0
    interrupted = False
    i = 0

    try:
        while i < total and not interrupted:
            r = df.iloc[i]

            # Skip jika sudah ada latlon
            if pd.notna(r[col_lat]) and pd.notna(r[col_lon]):
                print(f"▶ {i+1}/{total}: SKIP (sudah ada latlon)")
                skipped += 1
                i += 1
                continue

            # Skip jika alamat kosong
            alamat = str(r[col_alamat]).strip() if pd.notna(r[col_alamat]) else ""
            if alamat == "" or _missing(r[col_alamat]):
                print(f"▶ {i+1}/{total}: SKIP (alamat kosong)")
                skipped += 1
                i += 1
                continue

            nmusaha = str(r[col_nmusaha]).strip() if pd.notna(r[col_nmusaha]) else ""
            print(f"▶ {i+1}/{total}: {nmusaha} - {alamat[:50]}")
            processed += 1

            found = False

            # =========================================================
            # PENCARIAN 1: ALAMAT SAJA
            # =========================================================
            q_alamat = alamat
            try:
                box = wait.until(EC.presence_of_element_located((By.ID, "UGojuc")))
                box.clear()
                box.send_keys(q_alamat + Keys.ENTER)

                try:
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.hfpxzc")))
                    time.sleep(3)
                except Exception:
                    time.sleep(3)

                url = driver.current_url
                lat, lon = extract_coordinates(url)

                # CEK: Direct Point (ada elemen h1.DUwDvf)
                if is_direct_point(driver) and lat and lon:
                    if is_in_tabalong(lat, lon):
                        address = get_place_address(driver)
                        
                        df.at[i, col_lat] = lat
                        df.at[i, col_lon] = lon
                        df.at[i, col_gmaps] = address
                        
                        print(f"  ✅ DIRECT ALAMAT (dalam Tabalong): {lat:.5f}, {lon:.5f}")
                        print(f"     Alamat: {address[:80]}")
                        found = True
                    else:
                        print(f"  ⚠️  Direct point di luar Tabalong: {lat:.5f}, {lon:.5f}")

                # CEK: List Results (tidak ada h1.DUwDvf)
                elif not found:
                    try:
                        results = driver.find_elements(By.CSS_SELECTOR, "a.hfpxzc")
                        
                        if len(results) > 0:
                            # Klik hasil pertama
                            first_result = results[0]
                            first_result.click()
                            time.sleep(3)

                            url = driver.current_url
                            lat, lon = extract_coordinates(url)

                            if lat and lon:
                                if is_in_tabalong(lat, lon):
                                    address = get_place_address(driver)
                                    
                                    df.at[i, col_lat] = lat
                                    df.at[i, col_lon] = lon
                                    df.at[i, col_gmaps] = address
                                    
                                    print(f"  ✅ LIST ALAMAT #1 (dalam Tabalong): {lat:.5f}, {lon:.5f}")
                                    print(f"     Alamat: {address[:80]}")
                                    found = True
                                else:
                                    print(f"  ⚠️  List #1 di luar Tabalong: {lat:.5f}, {lon:.5f}")
                        else:
                            print("  ⚠️  Tidak ada hasil list")
                    except Exception as e:
                        print(f"  ⚠️  Error list: {str(e)[:50]}")

            except ElementNotInteractableException:
                driver, wait, i = restart_and_skip(
                    driver, wait, i, total,
                    reason="ElementNotInteractableException (pencarian 1)",
                    mark="MANUAL - ElementNotInteractable (pencarian 1)"
                )
                continue
            except WebDriverException as e:
                driver, wait, i = restart_and_skip(
                    driver, wait, i, total,
                    reason=f"WebDriverException (pencarian 1): {str(e)[:80]}",
                    mark="MANUAL - WebDriverException (pencarian 1)"
                )
                continue
            except Exception as e:
                print(f"  ❌ Error pencarian 1: {str(e)[:50]}")

            # =========================================================
            # PENCARIAN 2: ALAMAT + "TABALONG" (jika belum ketemu atau keluar wilayah)
            # =========================================================
            if not found:
                q_tabalong = f"{alamat} tabalong"
                try:
                    box = wait.until(EC.presence_of_element_located((By.ID, "UGojuc")))
                    box.clear()
                    box.send_keys(q_tabalong + Keys.ENTER)

                    try:
                        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.hfpxzc")))
                        time.sleep(3)
                    except Exception:
                        time.sleep(3)

                    url = driver.current_url
                    lat, lon = extract_coordinates(url)

                    # CEK: Direct Point (ada elemen h1.DUwDvf)
                    if is_direct_point(driver) and lat and lon:
                        if is_in_tabalong(lat, lon):
                            address = get_place_address(driver)
                            
                            df.at[i, col_lat] = lat
                            df.at[i, col_lon] = lon
                            df.at[i, col_gmaps] = address
                            
                            print(f"  ✅ DIRECT TABALONG: {lat:.5f}, {lon:.5f}")
                            print(f"     Alamat: {address[:80]}")
                            found = True
                        else:
                            print(f"  ❌ Direct tabalong tetap di luar: {lat:.5f}, {lon:.5f}")

                    # CEK: List Results
                    elif not found:
                        try:
                            results = driver.find_elements(By.CSS_SELECTOR, "a.hfpxzc")
                            
                            if len(results) > 0:
                                first_result = results[0]
                                first_result.click()
                                time.sleep(3)

                                url = driver.current_url
                                lat, lon = extract_coordinates(url)

                                if lat and lon and is_in_tabalong(lat, lon):
                                    address = get_place_address(driver)
                                    
                                    df.at[i, col_lat] = lat
                                    df.at[i, col_lon] = lon
                                    df.at[i, col_gmaps] = address
                                    
                                    print(f"  ✅ LIST TABALONG #1: {lat:.5f}, {lon:.5f}")
                                    print(f"     Alamat: {address[:80]}")
                                    found = True
                                else:
                                    print(f"  ❌ List tabalong #1 tetap di luar atau tidak ada koordinat")
                            else:
                                print(f"  ❌ Tidak ada hasil untuk '{alamat} tabalong'")
                        except Exception as e:
                            print(f"  ❌ Error list tabalong: {str(e)[:50]}")

                except ElementNotInteractableException:
                    driver, wait, i = restart_and_skip(
                        driver, wait, i, total,
                        reason="ElementNotInteractableException (pencarian 2)",
                        mark="MANUAL - ElementNotInteractable (pencarian 2)"
                    )
                    continue
                except WebDriverException as e:
                    driver, wait, i = restart_and_skip(
                        driver, wait, i, total,
                        reason=f"WebDriverException (pencarian 2): {str(e)[:80]}",
                        mark="MANUAL - WebDriverException (pencarian 2)"
                    )
                    continue
                except Exception as e:
                    print(f"  ❌ Error pencarian 2: {str(e)[:50]}")

            if not found:
                print(f"  ❌ TIDAK KETEMU untuk baris ini")

            time.sleep(sleep_time)
            i += 1

    except KeyboardInterrupt:
        print("\n⚠ Progres diinterupsi! Menyimpan progress...")
        interrupted = True
    finally:
        safe_quit(driver)
        print(f"\n✔ Processed: {processed}, Skipped: {skipped}, Total selesai: {i}/{total}")

    return df


# USAGE

In [29]:
# Usage
sbrdup = isi_latlong(
    sbrdup, 
    "nama_usaha", "alamat", "nmdesa", "nmkec", 
    "latitude", "longitude", "hasil_maps"
)

▶ 1/500: SKIP (sudah ada latlon)
▶ 2/500: SKIP (sudah ada latlon)
▶ 3/500: SKIP (sudah ada latlon)
▶ 4/500: SKIP (sudah ada latlon)
▶ 5/500: SKIP (sudah ada latlon)
▶ 6/500: SKIP TOTAL (keberadaan_usaha != 1)
▶ 7/500: CAKRA DENTA AGUNG PERTIWI, PT -  
  ⚠ Tidak ada hasil pencarian langsung
▶ 7/500: STOP setelah logika 0 (desa/kec kosong)
▶ 8/500: SKIP (sudah ada latlon)
▶ 9/500: SKIP (sudah ada latlon)
▶ 10/500: SKIP (sudah ada latlon)
▶ 11/500: SKIP (sudah ada latlon)
▶ 12/500: SKIP (sudah ada latlon)
▶ 13/500: SKIP (sudah ada latlon)
▶ 14/500: SKIP (sudah ada latlon)
▶ 15/500: SKIP (sudah ada latlon)
▶ 16/500: SKIP (sudah ada latlon)
▶ 17/500: KECAMATAN MURUNG PUDAK -  
  ⚠ Tidak ada hasil pencarian langsung
▶ 17/500: STOP setelah logika 0 (desa/kec kosong)
▶ 18/500: SKIP (sudah ada latlon)
▶ 19/500: SKIP (sudah ada latlon)
▶ 20/500: SKIP (sudah ada latlon)
▶ 21/500: SKIP (sudah ada latlon)
▶ 22/500: SKIP (sudah ada latlon)
▶ 23/500: SKIP (sudah ada latlon)
▶ 24/500: SKIP (sudah ada 

In [10]:
# Usage
sbrdup = isi_latlongdua(
    sbrdup, 
    "nama_usaha", "alamat", "nmdesa", "nmkec", 
    "latitude", "longitude", "hasil_maps"
)

▶ 1/500: AL AMIIN - Jalan Transpir Tarip
  ⚠️  Tidak ada hasil list
  ❌ Tidak ada hasil untuk 'Jalan Transpir Tarip tabalong'
  ❌ TIDAK KETEMU untuk baris ini
▶ 2/500: AL AMIIN - Jalan Transpir Tarip
  ⚠️  Tidak ada hasil list

⚠ Progres diinterupsi! Menyimpan progress...

✔ Processed: 2, Skipped: 0, Total selesai: 1/500


# injeksi baris ke x

In [90]:
x = 951  # contoh: baris ke-10 (posisi, 0-based)
sbrdup.iloc[x-1]

nomor                                                          951
Petugas                           Gilang Wahyu Prasetyo, S.Tr.Stat
idsbr                                                      2801720
nama_usaha                                    BUMDES KAYUH BAIMBAI
nama_komersial_usaha                                           NaN
alamat                                      JL BASUKI RAHMAT KM 17
nama_sls                                                       NaN
kodepos                                                        NaN
nomor_telepon                                          85249428346
nomor_whatsapp                                                 NaN
email                             bumdeskayuhbaimbai@mailnesia.com
website                                                        NaN
latitude                                                       NaN
longitude                                                      NaN
keberadaan_usaha                                              

In [92]:
#ganti baris ke-x dengan nilai yang mau diinjeksi

sbrdup.iloc[x, sbrdup.columns.get_indexer(['hasil_maps','latitude','longitude'])] = [
    "Kantor Desa Mahe Seberang",
    -2.048155802266606, 115.45539771581664
]


# hapus kolom tertentu yang kolom tertentunya bernilai tertentu

In [ ]:
sbrdup.loc[sbrdup["hasil_maps"].fillna("").str.lower() == "Kantor Bupati Tabalong", ["latitude", "longitude", "hasil_maps"]] = pd.NA


# kosongkan latitude, longitude dan hasil_maps mulai dari x sampai akhir baris

In [ ]:
start = 117  # iloc 117 = baris ke-118

# ambil index label untuk range iloc 117 s/d akhir
idx = sbrdup.iloc[start:].index

# kosongkan 3 kolom itu
sbrdup.loc[idx, ["latitude", "longitude", "hasil_maps"]] = np.nan  # atau "" untuk hasil_maps kalau mau string kosong

print("dikosongkan:", len(idx))

# Save ke Lokal

In [30]:
sbrdup.to_excel(r"C:\Users\ACER\Downloads\undefined part 1.xlsx", index=False)
#sbrdup = pd.read_excel(r"C:\Users\ACER\Downloads\sbrdup1.xlsx")


# OTAK ATIK

In [ ]:
# Filter berdasarkan hasil_maps
mask = sbrdup1['hasil_maps'] == "Kantor Desa MABURAI kecamatan MURUNG PUDAK kabupaten tabalong"

# Update nilai
sbrdup1.loc[mask, 'latitude'] = -2.184373955978614
sbrdup1.loc[mask, 'longitude'] = 115.43967627578346
sbrdup1.loc[mask, 'hasil_maps'] = "Kantor Lurah Maburai"

# Verifikasi
print(f"Updated {mask.sum()} row(s)")
print(sbrdup1.loc[mask, ['latitude', 'longitude', 'hasil_maps']].head())


In [ ]:
# Ambil index 10 baris terakhir yang punya lat/lon terisi
filled = sbrdup[(sbrdup['latitude'].notna()) & (sbrdup['longitude'].notna())]
last_10_idx = filled.tail(6).index

# Hapus lat/lon/gmaps di 10 baris terakhir
sbrdup.loc[last_10_idx, ['latitude', 'longitude', 'hasil_maps']] = None

print(f"✔ Hapus lat/lon di {len(last_10_idx)} baris terakhir:")
print(last_10_idx.tolist())


# Kill Chrome

In [ ]:
import psutil, os

def kill_all_chrome_spawned_by_this_kernel():
    me = psutil.Process(os.getpid())
    procs = me.children(recursive=True)  # semua child dari kernel ipynb ini[web:192]
    targets = [p for p in procs if (p.name() or "").lower() in ("chrome.exe","chrome","chromedriver.exe","chromedriver")]
    for p in targets:
        try: p.kill()  # hard kill[web:192]
        except: pass
    psutil.wait_procs(targets, timeout=3)  # tunggu mati[web:192]

kill_all_chrome_spawned_by_this_kernel()